In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
os.environ['CUDA_VISIBLE_DEVICES']='5'
import pprint

import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
from torchsummary import summary

import _init_paths
from core.config import config
from core.config import update_config
from core.config import update_dir
from core.loss import JointsMSELoss
from core.function import validate
from utils.utils import create_logger

import dataset
import models

import quantize_dorefa
from quantize_iao import *
# from quantize_iao_uint import *  #对feature map进行uint对称量化

import numpy as np
# 保证所有数据能够显示，而不是用省略号表示，np.inf表示一个足够大的数
np.set_printoptions(threshold = np.inf) 

# # 若想不以科学计数显示:
# np.set_printoptions(suppress = True)

In [2]:
def bn_fuse_conv(bn_conv,device):
    # ******************** bn参数 *********************
    mean = bn_conv.running_mean
    std = torch.sqrt(bn_conv.running_var + bn_conv.eps)
    gamma = bn_conv.gamma
    beta = bn_conv.beta
    # ******************* conv参数 ********************
    w = bn_conv.weight
    w_fused = w.clone()
    if bn_conv.bias is not None:
        b = bn_conv.bias
    else:
        b = mean.new_zeros(mean.shape)
    b_fused = b.clone()
    # ******************* bn融合 *******************
    w_fused = w * (gamma / std).reshape([bn_conv.out_channels, 1, 1, 1])
    b_fused = beta + (b - mean) * (gamma / std)
    bn_fused_conv = QuantConv2d(bn_conv.in_channels,
                                         bn_conv.out_channels,
                                         bn_conv.kernel_size,
                                         stride=bn_conv.stride,
                                         padding=bn_conv.padding,
                                         dilation=bn_conv.dilation,
                                         groups=bn_conv.groups,
                                         bias=True,
                                         padding_mode=bn_conv.padding_mode,
                                         a_bits=config.QUANTIZATION.A_BITS,
                                         w_bits=config.QUANTIZATION.W_BITS,
                                         q_type=config.QUANTIZATION.Q_TYPE,
                                         q_level=config.QUANTIZATION.Q_LEVEL,
                                         device=device,
                                         quant_inference=True)
    bn_fused_conv.weight.data = w_fused
    bn_fused_conv.bias.data = b_fused
    bn_fused_conv.activation_quantizer.scale.copy_(bn_conv.activation_quantizer.scale)
    bn_fused_conv.activation_quantizer.zero_point.copy_(bn_conv.activation_quantizer.zero_point)
    bn_fused_conv.activation_quantizer.eps = bn_conv.activation_quantizer.eps
    bn_fused_conv.weight_quantizer.scale.copy_(bn_conv.weight_quantizer.scale)
    bn_fused_conv.weight_quantizer.zero_point.copy_(bn_conv.weight_quantizer.zero_point)
    bn_fused_conv.weight_quantizer.eps = bn_conv.weight_quantizer.eps
    return bn_fused_conv

def bn_fuse_deconv(bn_conv,device):
    # ******************** bn参数 *********************
    mean = bn_conv.running_mean
    std = torch.sqrt(bn_conv.running_var + bn_conv.eps)
    gamma = bn_conv.gamma
    beta = bn_conv.beta
    # ******************* conv参数 ********************
    w = bn_conv.weight
    w_fused = w.clone()
    if bn_conv.bias is not None:
        b = bn_conv.bias
    else:
        b = mean.new_zeros(mean.shape)
    b_fused = b.clone()
    # ******************* bn融合 *******************
    w_fused = w * (gamma / std).reshape([bn_conv.out_channels, 1, 1, 1])
    b_fused = beta + (b - mean) * (gamma / std)
    bn_fused_conv = QuantConvTranspose2d(bn_conv.in_channels,
                                         bn_conv.out_channels,
                                         bn_conv.kernel_size,
                                         stride=bn_conv.stride,
                                         padding=bn_conv.padding,
                                         output_padding=bn_conv.output_padding,
                                         dilation=bn_conv.dilation,
                                         groups=bn_conv.groups,
                                         bias=True,
                                         padding_mode=bn_conv.padding_mode,
                                         a_bits=config.QUANTIZATION.A_BITS,
                                         w_bits=config.QUANTIZATION.W_BITS,
                                         q_type=config.QUANTIZATION.Q_TYPE,
                                         q_level=config.QUANTIZATION.Q_LEVEL,
                                         device=device,
                                         quant_inference=True)
    bn_fused_conv.weight.data = w_fused
    bn_fused_conv.bias.data = b_fused
    bn_fused_conv.activation_quantizer.scale.copy_(bn_conv.activation_quantizer.scale)
    bn_fused_conv.activation_quantizer.zero_point.copy_(bn_conv.activation_quantizer.zero_point)
    bn_fused_conv.activation_quantizer.eps = bn_conv.activation_quantizer.eps
    bn_fused_conv.weight_quantizer.scale.copy_(bn_conv.weight_quantizer.scale)
    bn_fused_conv.weight_quantizer.zero_point.copy_(bn_conv.weight_quantizer.zero_point)
    bn_fused_conv.weight_quantizer.eps = bn_conv.weight_quantizer.eps
    return bn_fused_conv

In [3]:
def bn_fuse_module(module, device):
    for name, child in module.named_children():
        if isinstance(child, QuantBNFuseConv2d):
            bn_fused_conv = bn_fuse_conv(child, device)
            module._modules[name] = bn_fused_conv
        elif isinstance(child, QuantBNFuseConvTranspose2d):
            bn_fused_deconv = bn_fuse_deconv(child, device)
            module._modules[name] = bn_fused_deconv
        else:
            bn_fuse_module(child, device)


def model_bn_fuse(model, inplace=False):
    if not inplace:
        model = copy.deepcopy(model)
    device = next(model.parameters()).device
    bn_fuse_module(model,device)
    return model

In [4]:
def select_device(device='', apex=False, batch_size=None):
    # device = 'cpu' or '0' or '0,1,2,3'
    cpu_request = device.lower() == 'cpu'
    if device and not cpu_request:  # if device requested other than 'cpu'
        # os.environ['CUDA_VISIBLE_DEVICES'] = device  # set environment variable
        assert torch.cuda.is_available(), 'CUDA unavailable, invalid device %s requested' % device  # check availablity

    cuda = False if cpu_request else torch.cuda.is_available()
    if cuda:
        c = 1024 ** 2  # bytes to MB
        ng = torch.cuda.device_count()
        if ng > 1 and batch_size:  # check that batch_size is compatible with device_count
            assert batch_size % ng == 0, 'batch-size %g not multiple of GPU count %g' % (batch_size, ng)
        x = [torch.cuda.get_device_properties(i) for i in range(ng)]
        s = 'Using CUDA ' + ('Apex ' if apex else '')  # apex for mixed precision https://github.com/NVIDIA/apex
        for i in range(0, ng):
            if i == 1:
                s = ' ' * len(s)
            print("%sdevice%g _CudaDeviceProperties(name='%s', total_memory=%dMB)" %
                  (s, i, x[i].name, x[i].total_memory / c))
    else:
        print('Using CPU')

    print('')  # skip a line
    return torch.device('cuda:0' if cuda else 'cpu')

In [5]:
cfg='../experiments/coco/resnet50/mobile_quant_relu_int.yaml'   #MODEL_FILE:'output/weights_quan/int8_mobilenet8_relu_bnfuse_inference.pt' 生成 'output/weights_quan/float_mobilenetpose_nobn.pt'
update_config(cfg)
# cudnn related setting
cudnn.benchmark = config.CUDNN.BENCHMARK
torch.backends.cudnn.deterministic = config.CUDNN.DETERMINISTIC
torch.backends.cudnn.enabled = config.CUDNN.ENABLED

# for shufflenetv2
shufflenetv2_spec = {'0.5': ([4, 8, 4], [24, 48, 96, 192, 1024]),
                        '1.0': ([4, 8, 4], [24, 116, 232, 464, 1024]),
                        '1.5': ([4, 8, 4], [24, 176, 352, 704, 1024]),
                        '2.0': ([4, 8, 4], [24, 244, 488, 976, 2048])}
stages_repeats, stages_out_channels = shufflenetv2_spec['1.0']
print('models.'+config.MODEL.NAME+'.get_pose_net')
model = eval('models.'+config.MODEL.NAME+'.get_pose_net')(
        config, 
        stages_repeats, stages_out_channels,
        is_train=False
    )

models.pose_mobilenet_relu.get_pose_net
/home/ytwang/wyt_workspace/quantization/human-pose-estimation.pytorch/pose_estimation/../lib/core/config.py:196: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  exp_config = edict(yaml.load(f))


In [6]:
####################################### bnfuse model ############################################
bnfuse_model = eval('models.pose_mobilenet_relu_bnfuse.get_pose_net')(
    config, 
    stages_repeats, stages_out_channels,
    is_train=False
)

In [7]:
gpus = [int(i) for i in config.GPUS.split(',')]
device = select_device(config.GPUS, batch_size=config.TEST.BATCH_SIZE*len(gpus))

model = model.to(device)
# summary(model,input_size=(3, 256, 192))

Using CUDA device0 _CudaDeviceProperties(name='GeForce RTX 2080 Ti', total_memory=11019MB)



In [8]:
#print('*******************ori_model*******************\n', model)
if(config.QUANTIZATION.QUANT_METHOD == 1): # DoReFa
    quantize_dorefa.prepare(model, inplace=True, a_bits=config.QUANTIZATION.A_BITS, w_bits=config.QUANTIZATION.W_BITS, quant_inference=config.QUANTIZATION.QUANT_INFERENCE, is_activate=False)
else: #default quant_method == 0   IAO
    prepare(model, inplace=True, a_bits=config.QUANTIZATION.A_BITS, w_bits=config.QUANTIZATION.W_BITS,q_type=config.QUANTIZATION.Q_TYPE, q_level=config.QUANTIZATION.Q_LEVEL, device=device,#device=next(model.parameters()).device, 
                        weight_observer=config.QUANTIZATION.WEIGHT_OBSERVER, bn_fuse=config.QUANTIZATION.BN_FUSE, quant_inference=config.QUANTIZATION.QUANT_INFERENCE)
#print('\n*******************quant_model*******************\n', model)
# print('\n*******************Using quant_model in test*******************\n')

a_bits= 8 	w_bits= 8 	q_type= 0 	q_level= 0 	device= cuda:0 	weight_observer= 0 	bn_fuse= 1 	quant_inference= False


In [9]:
# if config.TEST.MODEL_FILE:
#     # logger.info('=> loading model from {}'.format(config.TEST.MODEL_FILE))
#     if(config.TEST.MODEL_FILE.split('/')[-1]=='checkpoint.pth.tar'):
#         model = torch.nn.DataParallel(model, device_ids=gpus).cuda()
#         #model.load_state_dict(torch.load(config.TEST.MODEL_FILE,map_location=torch.device('cuda'))['state_dict'])
#         model.load_state_dict(torch.load(config.TEST.MODEL_FILE,map_location=device)['state_dict'])
#         #torch.save(model.module.state_dict(), 'output/coco_quan/mobile_quant_relu_w8a8_bnfuse0/checkpoint_nomodule.pth.tar')
#     elif(config.TEST.MODEL_FILE.split('/')[-1]=='model_best.pth.tar'):  #multiGPU has model.module.
#         model = torch.nn.DataParallel(model, device_ids=gpus).cuda()
#         model.load_state_dict(torch.load(config.TEST.MODEL_FILE,map_location=device))
#     elif(config.TEST.MODEL_FILE.split('/')[-1]=='checkpoint_resave.pth.tar'):  #multiGPU has model.module.
#         model = torch.nn.DataParallel(model, device_ids=gpus).cuda()
#         model.load_state_dict(torch.load(config.TEST.MODEL_FILE,map_location=device))
#     else:  #final_state.pth.tar
#         model.load_state_dict(torch.load(config.TEST.MODEL_FILE,map_location=device))
#         model = torch.nn.DataParallel(model, device_ids=gpus).cuda()

In [10]:
######################################################################################################
# ********************* quant_bn_fused_model_inference **********************
model.to(device)
model_bn_fuse(model, inplace=True)  # bn融合
# print('\n*******************For inference bn_fuse quant_model*******************\n', model)
# ckpt = {'model': model.module.state_dict() if hasattr(model, 'module') else model.state_dict()}
# torch.save(ckpt, '../output/weights_quan/int8_mobilenet8_relu_bnfuse_inference.pt')
print('*******************For inference bn_fuse quant_model*******************')

*******************For inference bn_fuse quant_model*******************


In [11]:
print(config.TEST.MODEL_FILE)
model.load_state_dict(torch.load('../'+config.TEST.MODEL_FILE,map_location=device)['model'])  ##为什么还在'model'里面呀？
# model.load_state_dict(torch.load('../'+config.TEST.MODEL_FILE,map_location=device))  ##为什么还在'model'里面呀？
# model = torch.nn.DataParallel(model, device_ids=gpus).cuda()

output/weights_quan/int8_mobilenet8_relu_bnfuse_inference.pt


<All keys matched successfully>

In [12]:
remapped_state = {}
print('Model.state_dict:')
# ######################################## before #######################################
# for n,param_tensor in enumerate(model.state_dict()):
#     #打印 key value字典
#     print(n, param_tensor,'\t',model.state_dict()[param_tensor].size())
#     # if(n<5):
#     #     print(n, param_tensor,'\t',model.state_dict()[param_tensor].size())
#     #     # print(model.state_dict()[param_tensor])

# ######################################### after #######################################
# for n,param_tensor in enumerate(bnfuse_model.state_dict()):
#     #打印 key value字典
#     print(n, param_tensor,'\t',bnfuse_model.state_dict()[param_tensor].size())
#     # if(n<4):
#     #     print(n, param_tensor,'\t',model.state_dict()[param_tensor].size())
#     #     print(model.state_dict()[param_tensor])

Model.state_dict:


In [13]:
# ################################# 生成浮点权重 refactor后 #############################
# import re
# remapped_state = {}
# for n,state_key in enumerate(bnfuse_model.state_dict().keys()):
#     k = state_key.split('.') # pytorch  ['features', '0', '0', 'weight']
#     if(k[0]=='features'):
#         k[1]=str(int(k[1])+1)
#         k[-2]=str((int(k[2][-1])-1)*3)
#         k[2]='conv'
#         remapped_state_key=('.').join(k) #进行重映射
#     elif(k[0].startswith('deconv_layers')): #final_layer
#         number=3*int(k[0][-1])
#         remapped_state_key='deconv_layers.'+str(number)+'.'+k[-1] #weight/bias
#     elif(k[0]=='conv1'): #final_layer
#         remapped_state_key='features.0.0.'+k[-1] #weight/bias
#     else: #final_layer  conv2
#         remapped_state_key=state_key
#     # print(n, state_key, remapped_state_key, model.state_dict()[remapped_state_key].shape)
#     remapped_state[state_key]= model.state_dict()[remapped_state_key]    

In [14]:
# ################################# 生成浮点权重 refactor前 #############################
# remapped_state = {}
# for n,state_key in enumerate(bnfuse_model.state_dict().keys()):
#     k = state_key.split('.') # pytorch  ['features', '0', '0', 'weight']
#     if(k[0]!='final_layer'):
#         number = int(k[-2])//2*3 
#         # print(number)
#         k[-2]=str(number)
#         # print(k)
#         remapped_state_key=('.').join(k) #进行重映射
#     else: #final_layer
#         remapped_state_key=state_key
#     print(n, state_key, model.state_dict()[remapped_state_key].shape)
#     remapped_state[state_key]= model.state_dict()[remapped_state_key]   

In [16]:
################################### 量化步骤 ##############################
## 对称量化，因此zero_point均为0
def quantize_tensor(weight, bias, wscale, ascale, num_bits=8):
    qmin = -2**(num_bits-1)  #8bit [-128,127]
    qmax = 2**(num_bits-1) - 1
    q_weight= torch.round(weight/wscale).clamp_(qmin, qmax).type(torch.int32)
    # print('0:', torch.max(torch.round(weight/wscale)), torch.min(torch.round(weight/wscale)))
    q_bias= torch.round(bias/wscale.flatten()/ascale).type(torch.int32)  #bias没有进行截断
    return q_weight, q_bias

#反量化回浮点结果
def dequantize_tensor(q_weight, q_bias, wscale, ascale):
    return wscale*q_weight.float(), wscale.flatten()*ascale*q_bias.float()

In [17]:
############################################################## 定点结果导出 refactor后 ##############################################
remapped_state = {}
M_list = {}  #存储int推断时每个conv之后的*M scale
M_key=[]  #存储M_list中的索引键值
oscale_list={} #储存 浮点和整型的oscale
ascale_list={} #储存 浮点和整型的ascale
wscale_list={} #储存 浮点和整型的wscale

count=0
#导出权重和偏置至二进制文件中
print('Model.state_dict:')
for n,param_key in enumerate(model.state_dict()): # AttributeError: 'collections.OrderedDict' object has no attribute 'key',所以这里不使用model.state_dict().keys()
    #打印 key value字典
    # print(n, param_key,'\t',model.state_dict()[param_key].size())
    if(n==0): #if(n<4*2): if(n<=434):  n==16 or n==8
        # print(n, param_key,'\t',model.state_dict()[param_key].size()) #param_key: features.0.0.activation_quantizer.scale 
        ############################################## 以下得到浮点权重值 ###################################################
        if(param_key.split('.')[-1]=='weight'):  #最后一层final_layer没有bias,但最后一层本来就应该要单独设计的？
            layer_name=('.').join(param_key.split('.')[0:-1])
            next_layer=''
            # print(layer_name+'.weight')
            conv_weight=model.state_dict()[layer_name+'.weight'].detach().cpu()
            ascale=model.state_dict()[layer_name+'.activation_quantizer.scale'].detach().cpu()  #对称量化，因此zero_point为0
            wscale=model.state_dict()[layer_name+'.weight_quantizer.scale'].detach().cpu()  # 通道量化  torch.Size([16, 1, 1, 1])

            if(param_key.split('.')[0]!='final_layer'):
                conv_bias=model.state_dict()[layer_name+'.bias'].detach().cpu()  # final_layer其实是没有bias的
                tmp=param_key.split('.')[0:-1]
                tmp[-1]=str(int(tmp[-1])+2)
                next_layer=('.').join(tmp)
                print(param_key, next_layer)
            else:
                print('It is final_layer')
                print(conv_weight.shape)
            
            if((next_layer+'.activation_quantizer.scale') in model.state_dict()):
                oscale=model.state_dict()[next_layer+'.activation_quantizer.scale'].detach().cpu()  #存在relu的情况 包括网络第一层、InvertedResidual中包含relu的层、conv2以及deconv_layers
                # print('0:',next_layer, oscale)
            elif(param_key.split('.')[1]!='17' and next_layer!=''): #不存在relu的情况  此时是mobilenet 每个InvertedResidual模块的末尾（线性直通，无relu)   但features.17的oscale应该是conv2的ascale
                tmp2=param_key.split('.')[0:-1]
                tmp2[1]=str(int(tmp[1])+1)
                tmp2[-1]='0'
                next_layer=('.').join(tmp2)
                oscale=model.state_dict()[next_layer+'.activation_quantizer.scale'].detach().cpu()
                # print('1:',next_layer,oscale)
            elif(param_key.split('.')[1]=='17'): #features.17的oscale应该是final_layer的ascale
                oscale=model.state_dict()['conv2.0.activation_quantizer.scale'].detach().cpu()
                # print('2: conv2.0',oscale)
            else:  #final_layer 不需要oscale    next_layer=='' 没有relu的都不需要oscale
                oscale=torch.tensor([1.])
                # print('3: final_layer',oscale)
            print(conv_weight.shape, 'conv_bias=',conv_bias.shape,'\nascale=',ascale, ascale.shape, '\nwscale=',wscale.flatten(), wscale.shape, '\noscale=',oscale,oscale.shape) #wscale.flatten()

            ############################################## 以下进行浮点权重的量化，得到int权重和M ###################################################
            if(param_key.split('.')[0]!='final_layer'):
                #计算移位值  不同通道的wscale还相差蛮大的（e-02,e-07) 2**16好像还不够  所以先使用M来验证
                # if(param_key.split('.')[0]=='features'and param_key.split('.')[1]=='0'): #第一层的M需要特殊处理，将输入ascale放入图像预处理中实现，输入网络的数据直接是[-128,127] 似乎又不需要...
                #     M=wscale/oscale
                # else:
                #     M=wscale*ascale/oscale  #一开始的不同通道间M差距很大（e-03,e-09)， 后面就挺均匀的  
                M=wscale*ascale/oscale
                # M0=(M*2**16).type(torch.int32)
                # print('M=',M.flatten(),'  M0=',M0.flatten(),'   error=',((M-M0*2**(-16))/M).flatten()) #
                print('###: M=',M.flatten())

                print(conv_weight.shape, conv_bias.shape)
                #*********************************************************************************************************************************
                print('conv_weight:',conv_weight.squeeze(), 'conv_bias:',conv_bias.flatten())
                
                #计算权重和偏置int量化结果
                q_weight,q_bias = quantize_tensor(conv_weight, conv_bias, wscale, ascale, num_bits=8)
                print(q_weight.shape, q_bias.shape)
                print(q_weight.flatten(), q_bias.flatten())
                
                #反量化回浮点数的结果
                dq_weight, dq_bias = dequantize_tensor(q_weight, q_bias, wscale, ascale)
                # print(dq_weight, dq_bias)
                print('weight:', torch.mean(conv_weight-dq_weight), torch.max(conv_weight-dq_weight),'  bias', torch.mean(conv_bias-dq_bias), torch.max(conv_bias-dq_bias))
                # print(conv_bias-dq_bias)
            else: #final_layer
                #计算移位值  不同通道的wscale还相差蛮大的（e-02,e-07) 2**16好像还不够  所以先使用M来验证
                M=wscale*ascale  #一开始的不同通道间M差距很大（e-03,e-09)， 后面就挺均匀的  
                # M0=(M*2**16).type(torch.int32) #如果最后一层直接使用浮点数进行运算，则不用计算M0
                # print('M=',M.flatten(),'  M0=',M0.flatten(),'   error=',((M-M0*2**(16))/M).flatten()) #
                print('final_layer: M=',M.flatten())

                #计算权重和偏置int量化结果
                q_weight,q_bias = quantize_tensor(conv_weight, torch.zeros([17]), wscale, ascale, num_bits=8) #这儿的bias是上一层的bias,是没有用的
                # print(q_weight, q_bias)
                # print(q_bias.shape)
                #反量化回浮点数的结果
                dq_weight, dq_bias = dequantize_tensor(q_weight, q_bias, wscale, ascale)
                # print(dq_weight, dq_bias)
                print(torch.mean(conv_weight-dq_weight), torch.max(conv_weight-dq_weight))
                # print(conv_bias-dq_bias)

            # ################################################### 浮点权重导出 ################################################################
            # q_weight=conv_weight #浮点权重的导出，需要控制变量呀！！！
            # q_bias=conv_bias
            # ################################################################################################################################

            ############################################## 以下将int权重映射给bnfuse_model ###################################################
            k = param_key.split('.') # pytorch  ['features', '0', '0', 'weight']
            if(k[0]=='features' and k[1]=='0'): # 网络第一层  features.0. -> conv1
                remapped_state_key = 'conv1.weight'
                remapped_state[remapped_state_key]=q_weight
                remapped_state[remapped_state_key.replace('weight','bias')]=q_bias
            elif(k[0]=='features'): #除了第一层外的其他层
                k[1]=str(int(k[1])-1)
                number = int(k[-2])//3 + 1 
                # print(number)
                k[2]=k[2]+str(number)
                # k[3]='0' #现在不需要了
                # print(k)
                remapped_state_key=('.').join(k[0:3])+'.weight' #进行重映射
                # print(count, n, remapped_state_key, bnfuse_model.state_dict()[remapped_state_key].shape)
                remapped_state[remapped_state_key]=q_weight
                remapped_state[remapped_state_key.replace('weight','bias')]=q_bias
                # print(count, n, remapped_state_key.replace('weight','bias'), bnfuse_model.state_dict()[remapped_state_key.replace('weight','bias')].shape)
            elif(k[0]=='deconv_layers'): #deconv_layers0-5
                number = int(k[-2])//3
                remapped_state_key=k[0]+str(number)+'.weight' #进行重映射
                remapped_state[remapped_state_key]=q_weight
                remapped_state[k[0]+str(number)+'.bias']=q_bias
                # print(count, n, remapped_state_key.replace('weight','bias'), bnfuse_model.state_dict()[remapped_state_key.replace('weight','bias')].shape)
            elif(k[0]=='final_layer'): #final_layer
                remapped_state_key=param_key
                remapped_state[remapped_state_key]=q_weight
                # print(count, n, remapped_state_key, bnfuse_model.state_dict()[remapped_state_key].shape)
            else: #conv2 无需进行重映射
                remapped_state_key = 'conv2.weight'
                remapped_state[remapped_state_key]=q_weight
                remapped_state[remapped_state_key.replace('weight','bias')]=q_bias
            # print(n, state_key, model.state_dict()[remapped_state_key].shape)
            # remapped_state[state_key]= model.state_dict()[remapped_state_key]   
            if(remapped_state_key.split('.')[0]=='features'):
                M_key_name = '.'.join(remapped_state_key.split('.')[0:-1])
            else:
                M_key_name = remapped_state_key.split('.')[0]
            M_key.append(M_key_name)
            M_list[M_key_name] = torch.squeeze(M, dim=-1) #M.flatten() torch.Size([16]);  M通道量化torch.Size([16, 1, 1, 1]) -> 需要转换为torch.Size([16, 1, 1])
            oscale_list[M_key_name]=oscale  #储存 浮点和整形的Oscale 是一个值  类似这样oscale= tensor([0.9272]) torch.Size([1])
            ascale_list[M_key_name]=ascale  
            wscale_list[M_key_name]=torch.squeeze(M, dim=-1)  
            count += 1

#保存M结果
print(M_key)
# np.save('../output/weights_quan/M_refactor.npy', M_list)
# np.save('../output/weights_quan/oscale.npy', oscale_list)
# np.save('../output/weights_quan/ascale.npy', ascale_list)
# np.save('../output/weights_quan/wscale.npy', wscale_list)


Model.state_dict:
features.0.0.weight features.0.2
torch.Size([16, 3, 3, 3]) conv_bias= torch.Size([16]) 
ascale= tensor([0.0207]) torch.Size([1]) 
wscale= tensor([3.2806e-02, 3.5383e-07, 6.9110e-03, 1.2938e-02, 1.1094e-02, 4.6062e-02,
        5.6726e-02, 4.9625e-02, 2.8893e-02, 2.8007e-05, 1.7617e-02, 1.5632e-06,
        2.1120e-02, 2.3379e-07, 1.3260e-06, 4.4465e-02]) torch.Size([16, 1, 1, 1]) 
oscale= tensor([0.9272]) torch.Size([1])
###: M= tensor([7.3260e-04, 7.9017e-09, 1.5433e-04, 2.8893e-04, 2.4775e-04, 1.0286e-03,
        1.2668e-03, 1.1082e-03, 6.4523e-04, 6.2544e-07, 3.9341e-04, 3.4910e-08,
        4.7164e-04, 5.2209e-09, 2.9611e-08, 9.9298e-04])
torch.Size([16, 3, 3, 3]) torch.Size([16])
conv_weight: tensor([[[[-1.7921e-01,  7.3493e-01,  2.8327e-01],
          [ 3.1658e-02,  1.0855e+00,  2.7640e-01],
          [ 1.1607e-01,  5.1940e-01,  3.0053e-01]],

         [[-2.2042e-01, -4.3684e-01, -1.0990e+00],
          [-1.3602e+00, -1.8952e+00, -4.0465e+00],
          [-6.7470e-0

In [18]:
############################################################## 定点结果导出 refactor前 ##############################################
# remapped_state = {}
# M_list = {}
# count=0
# #导出权重和偏置至二进制文件中
# print('Model.state_dict:')
# for n,param_key in enumerate(model.state_dict()): # AttributeError: 'collections.OrderedDict' object has no attribute 'key',所以这里不使用model.state_dict().keys()
#     #打印 key value字典
#     # print(n, param_key,'\t',model.state_dict()[param_key].size())
#     if(n<=434): #if(n<4*2): if(n<=434):
#         # print(n, param_key,'\t',model.state_dict()[param_key].size()) #param_key: features.0.0.activation_quantizer.scale 
#         ############################################## 以下得到浮点权重值 ###################################################
#         if(param_key.split('.')[-1]=='weight'):  #最后一层final_layer没有bias,但最后一层本来就应该要单独设计的？
#             layer_name=('.').join(param_key.split('.')[0:-1])
#             next_layer=''
#             # print(layer_name+'.weight')
#             conv_weight=model.state_dict()[layer_name+'.weight'].detach().cpu()
#             ascale=model.state_dict()[layer_name+'.activation_quantizer.scale'].detach().cpu()  #对称量化，因此zero_point为0
#             wscale=model.state_dict()[layer_name+'.weight_quantizer.scale'].detach().cpu()  # 通道量化  torch.Size([16, 1, 1, 1])

#             if(param_key.split('.')[0]!='final_layer'):
#                 conv_bias=model.state_dict()[layer_name+'.bias'].detach().cpu()  # final_layer其实是没有bias的
#                 tmp=param_key.split('.')[0:-1]
#                 tmp[-1]=str(int(tmp[-1])+2)
#                 next_layer=('.').join(tmp)
#             else:
#                 print('It is final_layer')
#                 print(conv_weight.shape)
            
#             if((next_layer+'.activation_quantizer.scale') in model.state_dict()):
#                 # print('0:',next_layer)
#                 oscale=model.state_dict()[next_layer+'.activation_quantizer.scale'].detach().cpu()  #存在relu的情况
#             elif(param_key.split('.')[1]!='17' and next_layer!=''): #不存在relu的情况  此时是mobilenet InvertedResidual模块的末尾
#                 tmp2=param_key.split('.')[0:-1]
#                 tmp2[1]=str(int(tmp[1])+1)
#                 tmp2[-1]='0'
#                 next_layer=('.').join(tmp2)
#                 # print('1:',next_layer)
#                 oscale=model.state_dict()[next_layer+'.activation_quantizer.scale'].detach().cpu()
#             elif(param_key.split('.')[1]!='17'):
#                 oscale=model.state_dict()['conv2.0.activation_quantizer.scale'].detach().cpu()
#             # print(conv_weight.shape, 'conv_bias=',conv_bias.shape,'\nascale=',ascale, ascale.shape, '\nwscale=',wscale.flatten(),wscale.shape, '\noscale=',oscale,oscale.shape)

#             ############################################## 以下进行浮点权重的量化，得到int权重和M ###################################################
#             if(param_key.split('.')[0]!='final_layer'):
#                 #计算移位值  不同通道的wscale还相差蛮大的（e-02,e-07) 2**16好像还不够  所以先使用M来验证
#                 M=wscale*ascale/oscale  #一开始的不同通道间M差距很大（e-03,e-09)， 后面就挺均匀的  
#                 M0=(M*2**16).type(torch.int32)
#                 # print('M=',M.flatten(),'  M0=',M0.flatten(),'   error=',((M-M0*2**(16))/M).flatten()) #

#                 #计算权重和偏置int量化结果
#                 q_weight,q_bias = quantize_tensor(conv_weight, conv_bias, wscale, ascale, num_bits=8)
#                 # print(q_weight, q_bias)
#                 # print(q_bias.shape)
#                 #反量化回浮点数的结果
#                 dq_weight, dq_bias = dequantize_tensor(q_weight, q_bias, wscale, ascale)
#                 # print(dq_weight, dq_bias)
#                 print(torch.mean(conv_weight-dq_weight), torch.max(conv_weight-dq_weight))
#                 # print(conv_bias-dq_bias)
#             else: #final_layer
#                 #计算移位值  不同通道的wscale还相差蛮大的（e-02,e-07) 2**16好像还不够  所以先使用M来验证
#                 M=wscale*ascale  #一开始的不同通道间M差距很大（e-03,e-09)， 后面就挺均匀的  
#                 # M0=(M*2**16).type(torch.int32) #如果最后一层直接使用浮点数进行运算，则不用计算M0
#                 # print('M=',M.flatten(),'  M0=',M0.flatten(),'   error=',((M-M0*2**(16))/M).flatten()) #

#                 #计算权重和偏置int量化结果
#                 q_weight,q_bias = quantize_tensor(conv_weight, torch.zeros([17]), wscale, ascale, num_bits=8) #这儿的bias是上一层的bias,是没有用的
#                 # print(q_weight, q_bias)
#                 # print(q_bias.shape)
#                 #反量化回浮点数的结果
#                 dq_weight, dq_bias = dequantize_tensor(q_weight, q_bias, wscale, ascale)
#                 # print(dq_weight, dq_bias)
#                 print(torch.mean(conv_weight-dq_weight), torch.max(conv_weight-dq_weight))
#                 # print(conv_bias-dq_bias)

                
#             ############################################## 以下将int权重映射给bnfuse_model ###################################################
#             k = param_key.split('.') # pytorch  ['features', '0', '0', 'weight']
#             if(k[0]!='final_layer'):
#                 number = int(k[-2])//3*2 
#                 # print(number)
#                 k[-2]=str(number)
#                 # print(k)
#                 remapped_state_key=('.').join(k) #进行重映射
#                 # print(count, n, remapped_state_key, bnfuse_model.state_dict()[remapped_state_key].shape)
#                 remapped_state[remapped_state_key]=q_weight
#                 remapped_state[remapped_state_key.replace('weight','bias')]=q_bias
#                 # print(count, n, remapped_state_key.replace('weight','bias'), bnfuse_model.state_dict()[remapped_state_key.replace('weight','bias')].shape)
#             else: #final_layer
#                 remapped_state_key=param_key
#                 remapped_state[remapped_state_key]=q_weight
#                 # print(count, n, remapped_state_key, bnfuse_model.state_dict()[remapped_state_key].shape)
#             # print(n, state_key, model.state_dict()[remapped_state_key].shape)
#             # remapped_state[state_key]= model.state_dict()[remapped_state_key]   
#             M_list[remapped_state_key] = M.flatten()
#             count += 1

# #保存M结果
# np.save('../output/weights_quan/M.npy', M_list)

In [21]:
print(len(M_key), M_key)
#读取M结果
# M_load = np.load('../output/weights_quan/M_refactor.npy', allow_pickle=True) #M_list
# # print('M_list=',M_load.item())
# print('conv1', M_load.item()['conv1'])#, M_load.item()[key])
# for n,key in enumerate(M_key):
#     if(n<=150):
#         print(n, key, M_load.item()[key].shape)#, M_load.item()[key])

# #读取oscale结果
# ascale_load = np.load('../output/weights_quan/ascale.npy', allow_pickle=True) 
# # print('M_list=',M_load.item())
# for n,key in enumerate(M_key):
#     if(n<=150):
#         print(n, key, ascale_load.item()[key], ascale_load.item()[key]*127.5)  #ascale_load.item()[key]*127.5 浮点结果

#读取oscale结果
oscale_load = np.load('../output/weights_quan/oscale.npy', allow_pickle=True) 
# print('M_list=',M_load.item())
for n,key in enumerate(M_key):
    if(n<=150):
        print(n, key, oscale_load.item()[key])

59 ['conv1', 'features.0.conv1', 'features.0.conv2', 'features.1.conv1', 'features.1.conv2', 'features.1.conv3', 'features.2.conv1', 'features.2.conv2', 'features.2.conv3', 'features.3.conv1', 'features.3.conv2', 'features.3.conv3', 'features.4.conv1', 'features.4.conv2', 'features.4.conv3', 'features.5.conv1', 'features.5.conv2', 'features.5.conv3', 'features.6.conv1', 'features.6.conv2', 'features.6.conv3', 'features.7.conv1', 'features.7.conv2', 'features.7.conv3', 'features.8.conv1', 'features.8.conv2', 'features.8.conv3', 'features.9.conv1', 'features.9.conv2', 'features.9.conv3', 'features.10.conv1', 'features.10.conv2', 'features.10.conv3', 'features.11.conv1', 'features.11.conv2', 'features.11.conv3', 'features.12.conv1', 'features.12.conv2', 'features.12.conv3', 'features.13.conv1', 'features.13.conv2', 'features.13.conv3', 'features.14.conv1', 'features.14.conv2', 'features.14.conv3', 'features.15.conv1', 'features.15.conv2', 'features.15.conv3', 'features.16.conv1', 'feature

In [20]:
#导入量化后的int权重  但由于模型中的weight/bias参数仍是float32类型的，因此保存时还是会保存成float32.
print(remapped_state.keys())
bnfuse_model.load_state_dict(remapped_state)
#修改权重数据的类型为int32
# for n,param_key in enumerate(bnfuse_model.state_dict()):
#     #打印 key value字典
#     print(n, param_key,'\t',bnfuse_model.state_dict()[param_key].size())
#     eval('bnfuse_model.'+param_key+'.data.type(torch.int32)')

# for name, module in bnfuse_model.named_modules():
#     if type(module) in [torch.nn.Conv2d, torch.nn.ConvTranspose2d]:
#         print(name, module)
#         if(name=='final_layer'): #没有bias
#             eval('bnfuse_model.'+name+'.weight.data.type(torch.int32)')
#         else: #conv2d, ConvTranspose2d
#             eval('bnfuse_model.'+name+'.weight.data.type(torch.int32)')
#             eval('bnfuse_model.'+name+'.bias.data.type(torch.int32)')

dict_keys(['conv1.weight', 'conv1.bias', 'features.0.conv1.weight', 'features.0.conv1.bias', 'features.0.conv2.weight', 'features.0.conv2.bias', 'features.1.conv1.weight', 'features.1.conv1.bias', 'features.1.conv2.weight', 'features.1.conv2.bias', 'features.1.conv3.weight', 'features.1.conv3.bias', 'features.2.conv1.weight', 'features.2.conv1.bias', 'features.2.conv2.weight', 'features.2.conv2.bias', 'features.2.conv3.weight', 'features.2.conv3.bias', 'features.3.conv1.weight', 'features.3.conv1.bias', 'features.3.conv2.weight', 'features.3.conv2.bias', 'features.3.conv3.weight', 'features.3.conv3.bias', 'features.4.conv1.weight', 'features.4.conv1.bias', 'features.4.conv2.weight', 'features.4.conv2.bias', 'features.4.conv3.weight', 'features.4.conv3.bias', 'features.5.conv1.weight', 'features.5.conv1.bias', 'features.5.conv2.weight', 'features.5.conv2.bias', 'features.5.conv3.weight', 'features.5.conv3.bias', 'features.6.conv1.weight', 'features.6.conv1.bias', 'features.6.conv2.weigh

<All keys matched successfully>

In [22]:
ckpt = {'model': bnfuse_model.module.state_dict() if hasattr(bnfuse_model, 'module') else bnfuse_model.state_dict()}
# torch.save(ckpt, '../output/weights_quan/float_mobilenetpose_nobn_refactor_likeint5_8.pt')
# torch.save(ckpt, '../output/weights_quan/float_mobilenetpose_nobn_refactor.pt')
torch.save(ckpt, '../output/weights_quan/int_mobilenetpose_nobn_refactor.pt')

In [21]:
###验证导出的权重是否正确
######################以下是bnfuse后的权重，与bnfuse_model.pt对应######################
###格式： [out_ch,in_ch,kernel_size1,kernel_size0]
###第一个是普通卷积
weight0_shape=[8,3,3,3]
bias0_shape=[8]
###接下来是深度可分离卷积
weight1_shape=[8,8,1,1]
bias1_shape=[8]
weight2_shape=[8,1,3,3]
bias2_shape=[8]
weight3_shape=[4,8,1,1]
bias3_shape=[4]

list=[[8,3,3,3],[8],[8,8,1,1],[8],[8,1,3,3],[8],[4,8,1,1],[4]]

#####################给的bin文件是权重重排后的结果######################
###格式： [kernel_size1,kernel_size0,out_ch,in_ch]
#以下可以将.bin文件中的权重返回.pt的数据排列，以便进行数据比对
result_path='./'
filename=['0_weight','0_bias','1_weight','1_bias','2_weight','2_bias','3_weight','3_bias']
for i in range(4*2):
    b=np.fromfile(result_path+filename[i]+'.bin',dtype=np.float32) ##一维的数据排列 ->[kernel_size1*kernel_size0*out_ch*in_ch]
    if(i==1):
        if(filename[i].split('_')[-1]=='weight'): #是weight
            b=b.reshape(list[i][2]*list[i][3],list[i][0],list[i][1]) #->[kernel_size1*kernel_size0,out_ch,in_ch]
            print(b.shape)
            b=b.transpose(1,2,0) #->[out_ch,in_ch,kernel_size1*kernel_size0]
            print(b.shape)
            b=b.reshape(list[i][0],list[i][1],list[i][2],list[i][3]) #->[out_ch,in_ch,kernel_size1,kernel_size0]
            print(b.shape)
            print(b)
        else: #否则是bias 无需转换，直接打印
            print(b)



FileNotFoundError: [Errno 2] No such file or directory: './0_weight.bin'

In [22]:
#b=np.fromfile(result_path+filename+'.bin',dtype=np.float32)
# b=np.loadtxt(result_path+filename+'.txt', dtype=np.float32, delimiter='  ')
# print(b.shape)
# b=b.reshape(9,8,3)
# print(b.shape)
# b=b.transpose(1,2,0) #[9,8,3]->[8,3,9]
# print(b.shape)
# b=b.reshape(8,3,3,3)
# print(b.shape)
# print(b)

# print(a.shape)
# print(a.shape[0],a.shape[1],a.shape[2],a.shape[3])
# #a_resize=a.reshape([8,3,-1])
# a_reshape=a.reshape([a.shape[0],a.shape[1],-1])
# print(a_reshape.shape)
# a_reshape=a_reshape.transpose(2,0,1) #[8,3,9]->[9,8,3]
# print(a_reshape.shape)

In [9]:
#a.reshape(8,3,9).transpose(2,1,0).shape #一次可以置换三个
#a.swapaxes(1,2)  #swapaxes只能两两置换 对于swapaxes来说，括号内的两参数，交换位置和不交换，实际结果相同。

# import cv2
# image=cv2.imread('/home/ytwang/dataset/VOC/images/val/000001.jpg')
# cv2.imshow("image",image)
# cv2.waitKey(0)
#cv2.destroyAllWindows()

In [10]:
##################################### 权重、feature map导出 ##################################
x0=torch.randn(1,3,352,256)
print(x0.shape) #[1,3,352,256]
x=x0.numpy().squeeze()
print(x.shape) #(3, 352, 256)
x=x.transpose(1,2,0)
print(x.shape) #(352,256,3)
#x.astype(np.float32).tofile(result_path+'input000001_352x256.bin') # 二进制文件导出
x=x.reshape([-1,x.shape[-1]]) #(90112,3)=(352*256,3)
print(x.shape)
#np.savetxt(result_path+'input.txt', x, fmt="%f", delimiter=',\t') # .txt文件导出



torch.Size([1, 3, 352, 256])
(3, 352, 256)
(352, 256, 3)
(90112, 3)


In [11]:
#b=np.fromfile(result_path+'input000001_352x256.bin',dtype=np.float32)
# b=np.loadtxt(result_path+'input.txt', dtype=np.float32, delimiter=',\t')
# print(b.shape)
# b=b.reshape(352,256,3).transpose(2,0,1)
# print(b.shape)
# b=b.reshape(1,3,352,256)
# print(b.shape)
# b=torch.tensor(b)
# type(b)
b=np.loadtxt(result_path+'input.txt', dtype=np.float32, delimiter=',\t')
print(b.shape)
print(x0.shape)



OSError: ./input.txt not found.

In [23]:
b=b.reshape(x0.shape[2],x0.shape[3],x0.shape[1]).transpose(2,0,1)
print(b.shape)
b=np.expand_dims(b,0) #[3,352,256]->[1,3,352,256]
print(b.shape)
x=torch.tensor(b)
print(type(x))

(3, 352, 256)
(1, 3, 352, 256)
<class 'torch.Tensor'>


In [13]:
# (x0==b).all()
print(x0[0][0][0]) #torch.Size([1, 3, 352, 256])

tensor([-6.81406e-01, -8.01516e-01,  3.28198e-01, -7.11460e-01, -9.71156e-01, -2.00248e-02, -1.63450e+00,  9.71171e-01, -7.00366e-01,  1.60246e+00, -1.78841e+00,  3.63713e-01,  9.74244e-01,  1.33411e+00, -3.02220e-01, -4.06555e-01,  1.37745e-01, -6.31540e-01, -1.46539e+00,  2.76620e-01,  1.23277e+00, -1.40946e+00,
        -1.51452e+00,  1.19671e-01,  4.42624e-01, -7.22908e-01, -4.04740e-01, -4.34604e-01,  2.06584e-01, -1.40587e+00,  5.70870e-01,  1.07140e-01,  1.61050e+00,  1.02770e+00, -3.53327e-01,  2.04177e-01,  2.31896e-02,  1.28927e+00, -5.48752e-01,  1.42098e-01,  9.03911e-01, -1.20936e+00,  9.63919e-01, -1.01060e+00,
         8.54119e-02, -1.53845e-01, -1.54839e+00, -6.02410e-02, -2.57630e+00,  3.93433e-01,  2.39229e-01, -1.21564e-01, -1.15327e+00,  1.26149e+00,  1.45040e+00, -9.07547e-01,  1.31109e+00, -1.65926e-02, -3.07028e-01,  6.83296e-01, -5.43891e-01,  6.85024e-01,  4.69522e-01,  1.76420e+00,  1.61510e+00,  1.25916e+00,
        -1.77150e-01,  2.47641e+00, -1.46249e+00, -2

In [36]:
# print(b)

In [18]:
####################################### 验证bnfuse后结果是否正确 #########################################
bn_weight=torch.tensor([
    0.9627532362937927,
    1.6811193227767944,
    1.0011911392211914,
    1.6239477396011353,
    1.3853583335876465,
    0.9575594663619995,
    1.325505256652832,
    0.9901668429374695])
bn_bias=torch.tensor([
    -24.751123428344727,
    16.553693771362305,
    -21.58427619934082,
    13.835719108581543,
    7.6066741943359375,
    12.691370010375977,
    24.215431213378906,
    -10.96066665649414])
bn_running_mean=torch.tensor([
    0.2264288365840912,
    -1.438944935798645,
    -0.012756695970892906,
    -1.1514368057250977,
    2.064279079437256,
    -1.000322699546814,
    1.4272160530090332,
    -0.01157035119831562])
bn_running_var=torch.tensor([
    0.018602905794978142,
    0.7236471176147461,
    0.0006818491965532303,
    0.586725652217865,
    1.5904737710952759,
    0.3611223101615906,
    0.7421475648880005,
    0.00004840613837586716])
bn_eps=torch.tensor([1E-4]).expand([8])

In [22]:
# bn.bias=
# bn.weight=
# bn.running_mean=
# bn.running_var=
# bn.eps=
afterfuse_b=bn_bias-bn_weight*bn_running_mean/(torch.sqrt(bn_running_var+bn_eps))
print(afterfuse_b)

b_bn = bn_bias - bn_weight.mul(bn_running_mean).div(torch.sqrt(bn_running_var + bn_eps))
print(b_bn)


tensor([-26.34514,  19.39717, -21.12751,  16.27666,   5.33914,  14.28511,  22.01961, -10.02023])
tensor([-26.34514,  19.39717, -21.12751,  16.27666,   5.33914,  14.28511,  22.01961, -10.02023])


In [ ]:
############################### torch_utils.py中的bu_fuse实现 ##########################
def fuse_conv_and_bn(conv, bn):
    # https://tehnokv.com/posts/fusing-batchnorm-and-conv/
    with torch.no_grad():
        # init
        fusedconv = torch.nn.Conv2d(conv.in_channels,
                                    conv.out_channels,
                                    kernel_size=conv.kernel_size,
                                    stride=conv.stride,
                                    padding=conv.padding,
                                    groups=conv.groups,
                                    bias=True)

        # prepare filters
        w_conv = conv.weight.clone().view(conv.out_channels, -1)
        w_bn = torch.diag(bn.weight.div(torch.sqrt(bn.eps + bn.running_var)))
        fusedconv.weight.copy_(torch.mm(w_bn, w_conv).view(fusedconv.weight.size()))

        # prepare spatial bias
        if conv.bias is not None:
            b_conv = conv.bias
        else:
            b_conv = torch.zeros(conv.weight.size(0))
        b_bn = bn.bias - bn.weight.mul(bn.running_mean).div(torch.sqrt(bn.running_var + bn.eps))
        fusedconv.bias.copy_(torch.mm(w_bn, b_conv.reshape(-1, 1)).reshape(-1) + b_bn)

        return fusedconv

In [ ]:
w_bn = torch.diag(bn.weight.div(torch.sqrt(bn.eps + bn.running_var)))